In [34]:
import matplotlib.pyplot as plt
import s3fs
import pandas as pd
import numpy as np
from itertools import product
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split,train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,mean_squared_error, mean_absolute_error, r2_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler
y_pred = np.loadtxt('y_pred.csv', delimiter=',')
y_test = np.loadtxt('y_test.csv', delimiter=',')
y_last = np.loadtxt('y_last.csv', delimiter=',')

In [37]:
y_last.shape

(383682,)

In [25]:
import numpy as np
from sklearn.metrics import confusion_matrix

def mse_r2_confusion_matrix(ground_truth, predictions, normal_range):
    """
    Calculate the Mean Squared Error (MSE), R-squared value (R2), and confusion matrix for two arrays, ignoring NaN values.
    The confusion matrix will be based on three categories: lower, normal, and higher.

    Parameters:
    ground_truth (numpy.ndarray): Ground truth array.
    predictions (numpy.ndarray): Predicted values array.
    normal_range (tuple): Normal range as a tuple (min, max).

    Returns:
    tuple: Three elements:
        - float: MSE value.
        - float: R-squared value.
        - numpy.ndarray: Confusion matrix.
    """
    # Ensure input arrays are numpy arrays
    ground_truth = np.asarray(ground_truth)
    predictions = np.asarray(predictions)

    if ground_truth.shape != predictions.shape:
        raise ValueError("The shapes of the ground truth and prediction arrays must be the same.")

    # Handle NaN values by masking
    mask = np.isnan(ground_truth) | np.isnan(predictions)
    gt = ground_truth[~mask]
    pred = predictions[~mask]

    # Calculate MSE
    mse = np.mean((gt - pred) ** 2)

    # Calculate R-squared
    mean_gt = np.mean(gt)
    tss = np.sum((gt - mean_gt) ** 2)
    rss = np.sum((gt - pred) ** 2)
    r2 = 1 - rss / tss

    # Normal range handling
    normal_min, normal_max = normal_range
    gt_category = np.where(gt < normal_min, 0, np.where(gt > normal_max, 2, 1))
    pred_category = np.where(pred < normal_min, 0, np.where(pred > normal_max, 2, 1))

    # Calculate confusion matrix
    cm = confusion_matrix(gt_category, pred_category, labels=[0, 1, 2])

    return mse, r2, cm

In [26]:
y_pred.iloc[:,0]

0         4.190452
1         4.536629
2         4.404006
3         4.288874
4         3.513833
            ...   
383677    4.426610
383678    3.513833
383679    3.938117
383680    4.271298
383681    4.011180
Name: 0, Length: 383682, dtype: float64

In [39]:
ItemID = 0
normal_ranges = [(3.5,5),(136,145),(98,106),(37,50),(0.7,1.3),(23,28),(7,13),(8,20),(0,200),(150,450)]

mse, r2, cm = mse_r2_confusion_matrix(y_test, y_pred[:,0], normal_ranges[ItemID])

print("Overall MAE MSE:", round(mse,3))
print("Overall MAE R-squared:", round(r2,3))
print("Overall MAE Confusion matrix:\n", cm)


valid_pos = ~np.isnan(y_last)
print('Test with the last value',len(y_last[valid_pos]))
mse, r2, cm = mse_r2_confusion_matrix(y_test, y_last, normal_ranges[ItemID])
print("last value prediction MSE per column:", round(mse,3))
print("last value prediction R-squared per column:", round(r2,3))
print("last value prediction Confusion matrix:\n", cm)

mse, r2, cm = mse_r2_confusion_matrix(y_test[valid_pos], y_pred[:,0][valid_pos], normal_ranges[ItemID])
print("MAE for those with last value MSE :", round(mse,3))
print("MAE for those with last value R-squared:", round(r2,3))
print("MAE for those with last value Confusion matrix:\n", cm)

Overall MAE MSE: 0.309
Overall MAE R-squared: -0.065
Overall MAE Confusion matrix:
 [[     0  26633    127]
 [     1 287554   2612]
 [     0  16095    718]]
Test with the last value 284042
last value prediction MSE per column: 0.227
last value prediction R-squared per column: 0.166
last value prediction Confusion matrix:
 [[  7330  13650    141]
 [ 15098 206947   8527]
 [   138   6883   5360]]
MAE for those with last value MSE : 0.3
MAE for those with last value R-squared: -0.1
MAE for those with last value Confusion matrix:
 [[     0  20994    127]
 [     1 227959   2612]
 [     0  11663    718]]


In [6]:
valid_pos.shape

(9314,)

In [8]:
currImpute.shape

(10000,)

In [5]:
mse_pred = mean_squared_error(currTest[valid_pos], currImpute[valid_pos])
r2_pred = r2_score(currTest[valid_pos], currImpute[valid_pos])
itemid="50971"
normalranges = {
  "Crt": (0.7, 1.3),
  "Plt": (150,450),
  "Hgb": (12, 18),
  "Wbc": (4, 11),
  "Ure": (8, 20),
  "50983": (136, 145),
  "Bic": (23, 28),
  "50971": (3.5, 5),
  "50902": (98, 106),
  "51221": (37,50),
}

bins = [-1,normalranges[f"{itemid}"][0],normalranges[f"{itemid}"][1],10000]
# Define labels for the categories
labels = ['Low', 'Medium', 'High']
y_test_cat = pd.cut(currTest[valid_pos],bins=bins, labels=labels).astype(str)
y_pred_cat = pd.cut(currImpute[valid_pos],bins=bins, labels=labels).astype(str)
print(itemid)
print("normal range", bins)
print("xgboost imputation mse ",round(mse_pred,3),"and r2: ", round(r2_pred,3))
cm_pred = confusion_matrix(y_test_cat,y_pred_cat,labels=labels)
print("confusion matrix for xgboost")
print(cm_pred)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [19]:
currImpute[valid_pos]

0       20.567667
1       19.296064
2       18.000000
3       13.000000
5       12.000000
          ...    
9995    32.000000
9996    15.000000
9997    11.000000
9998     7.000000
9999     6.000000
Name: 50971, Length: 8673, dtype: float64

In [20]:
currTest[valid_pos]

0       19.0
1       22.0
2       18.0
3       13.0
5       12.0
        ... 
9995    32.0
9996    15.0
9997    11.0
9998     7.0
9999     6.0
Name: 50971, Length: 8673, dtype: float64